In [ ]:
!pip install langchain langchain_openai langchain_experimental tabulate

In [ ]:
!pip install --upgrade openai

In [ ]:
import pandas as pd 

df = pd.read_csv('/Users/keshavsaraogi/Desktop/indorama/eureka-data/clean-csv/cleaned_finance_packaging.csv') 
df.head()

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.5
) 

agent_executer = create_csv_agent(
    llm, 
    '/Users/keshavsaraogi/Desktop/indorama/eureka-data/clean-csv/cleaned_finance_packaging.csv', 
    verbose=True,
    allow_dangerous_code=True,
    max_iterations=20,
    max_execution_time=60,
    handle_parsing_errors=True
)

In [ ]:
agent_executer.invoke("How many total company codes are there in the dataset? Name them all.")

In [ ]:
agent_executer.invoke("What is the sum of the debit balance in global currency? Explain how you arrived at the answer.")

In [ ]:
agent_executer.invoke("What is the sum of the total credit balance in global currency? Explain how you arrived at the answer.")

In [ ]:
agent_executer.invoke("Give me a list of the top 3 companies with the highest total credit balance in global currency. Explain how you arrived at the answer.")

In [ ]:
agent_executer.invoke("What is the sum of the ending balance of the ending balance in global currency? Explain how you arrived at the answer.")

IVL - 4,43,25,417.45
Bevpak - 1,54,32,474.98